In [1]:
pip install sentence-transformers

In [2]:
pip install pinecone-client

In [3]:
pip install openai==0.28

In [4]:
import openai
import pinecone
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
# Setting OpenAI API key
openai.api_key = "sk-I15f5H8te39SFxNBmAThT3Blb****yBG8gvkumIm5hp9****"

# Setting Pinecone API key
pinecone_api_key = "4fa45a4c-84cf-4194-****-355cec2d****"
pinecone_index_name = "qa_bot_index"

# Connecting to Pinecone
pinecone = pinecone.init(api_key="YOUR_PINECONE_API_KEY")

In [6]:
# Function to create or get Pinecone index
def create_or_get_index():
    if pinecone_index_name not in pinecone.list_indexes():
        pinecone.create_index(index_name=pinecone_index_name)
    return pinecone_index_name


In [7]:

# Function to insert vectors into Pinecone
def insert_vectors(vectors):
    index_name = create_or_get_index()
    pinecone.upsert_items(index_name=index_name, items=vectors)


In [8]:

# Function to search Pinecone index
def search_pinecone(query_vector, num_results=1):
    index_name = create_or_get_index()
    results = pinecone.query(index_name=index_name, query_vector=query_vector, top_k=num_results)
    return results

In [14]:

# Function to generate answer using OpenAI API
def generate_answer(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.5,
        max_tokens=150,
        n=1
    )
    return response.choices[0].text.strip()


In [10]:

# Loading a pre-trained sentence embedding model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [11]:
# Creating Convert to Vector function

def convert_to_vector(text):
    # Generate an embedding for the given text
    vector = embedding_model.encode(text)
    return vector


In [12]:

# Main function
def main():
    # Sample question
    question = "What is the capital of France?"

    # Using OpenAI API to generate a passage related to the question
    passage = generate_answer(question)

    # Converting the passage into a vector representation
    passage_vector = convert_to_vector(passage)  # You need to implement this function

    # Inserting the passage vector into Pinecone
    insert_vectors([(passage_vector, question)])

    # Giving a new question, finding the most relevant passage vector
    new_question = "Tell me about French history."
    new_question_vector = convert_to_vector(generate_answer(new_question))  # Implementing this function
    results = search_pinecone(new_question_vector)

    # Displaying the answer from the most relevant passage
    if results:
        answer = results[0]["data"]
        print(f"Answer to '{new_question}': {answer}")
    else:
        print("No relevant answer found.")



In [16]:
# Runing the main function
main()

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.